# RFM 客户分群分析
> By SherryAGI | 基于购买行为的客户细分

## 什么是 RFM？

| 指标 | 含义 | 计算方式 |
|------|------|----------|
| **R**ecency | 最近一次购买 | 距今天数 |
| **F**requency | 购买频率 | 购买次数 |
| **M**onetary | 消费金额 | 总消费额 |

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# SherryAGI - RFM Analysis
print('🚀 RFM 客户分群分析 by SherryAGI')

## 1. 加载数据

In [ ]:
# 示例：加载电商订单数据
# df = pd.read_csv('orders.csv')

# 模拟数据
np.random.seed(42)
n_customers = 1000

df = pd.DataFrame({
    'customer_id': np.random.randint(1, 201, n_customers),
    'order_date': pd.date_range('2025-01-01', periods=n_customers, freq='H'),
    'amount': np.random.exponential(100, n_customers)
})

df.head()

## 2. 计算 RFM 指标

In [ ]:
# 设定分析日期
analysis_date = df['order_date'].max() + pd.Timedelta(days=1)

# 计算 RFM
rfm = df.groupby('customer_id').agg({
    'order_date': lambda x: (analysis_date - x.max()).days,  # Recency
    'customer_id': 'count',  # Frequency (用 customer_id 计数)
    'amount': 'sum'  # Monetary
}).rename(columns={
    'order_date': 'recency',
    'customer_id': 'frequency',
    'amount': 'monetary'
})

rfm.head(10)

## 3. RFM 评分 (1-5分)

In [ ]:
# R 分数：越小越好（最近购买）
rfm['R_score'] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])

# F 分数：越大越好（购买频繁）
rfm['F_score'] = pd.qcut(rfm['frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])

# M 分数：越大越好（消费高）
rfm['M_score'] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

# 综合 RFM 分数
rfm['RFM_score'] = rfm['R_score'].astype(str) + rfm['F_score'].astype(str) + rfm['M_score'].astype(str)

rfm.head(10)

## 4. 客户分群

In [ ]:
def segment_customer(row):
    """基于 RFM 分数划分客户群"""
    r, f, m = int(row['R_score']), int(row['F_score']), int(row['M_score'])
    
    if r >= 4 and f >= 4 and m >= 4:
        return '🏆 冠军客户'
    elif r >= 4 and f >= 3:
        return '💎 忠诚客户'
    elif r >= 4 and f <= 2:
        return '🌟 新客户'
    elif r <= 2 and f >= 4:
        return '😴 沉睡客户'
    elif r <= 2 and f <= 2:
        return '💀 流失客户'
    else:
        return '📊 一般客户'

rfm['segment'] = rfm.apply(segment_customer, axis=1)

# 统计各群体数量
rfm['segment'].value_counts()

## 5. 可视化

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 客户分群饼图
rfm['segment'].value_counts().plot(kind='pie', ax=axes[0], autopct='%1.1f%%')
axes[0].set_title('客户分群分布 by SherryAGI')
axes[0].set_ylabel('')

# RFM 热力图
rfm_pivot = rfm.groupby(['R_score', 'F_score']).size().unstack()
sns.heatmap(rfm_pivot, annot=True, fmt='d', cmap='YlOrRd', ax=axes[1])
axes[1].set_title('R-F 分布热力图')

plt.tight_layout()
plt.show()

## 6. 营销建议

| 客户群 | 特征 | 营销策略 |
|--------|------|----------|
| 🏆 冠军客户 | 最近买、常买、买很多 | VIP 服务、专属优惠 |
| 💎 忠诚客户 | 最近买、常买 | 会员积分、推荐奖励 |
| 🌟 新客户 | 最近买、买得少 | 新人礼包、引导复购 |
| 😴 沉睡客户 | 很久没买、以前常买 | 唤醒邮件、限时折扣 |
| 💀 流失客户 | 很久没买、买得少 | 大额优惠券、调研原因 |

---
**By SherryAGI** | [DigitalTrans](https://github.com/AIB612/DigitalTrans)